In [ ]:
import topogenesis as tg
import numpy as np 
import pyvista as pv
import os
import trimesh as tm
import boolean_marching_cubes as bmc

In [ ]:
lattice_path = os.path.relpath('../data/voxelized_envelope_toy.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)
cube_lattice = envelope_lattice.boolean_marching_cubes()

In [ ]:
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# i'm trying to save directly to template folder from rhino but the end results is coming out very different from what i expected 

In [ ]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/voxelized_envelope_toy.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)
print(envelope_lattice.shape)
padded_env_arr = np.pad(envelope_lattice, 1, mode="constant", constant_values=False)
padded_minbound = envelope_lattice.minbound - envelope_lattice.unit
envelope_lattice = tg.to_lattice(padded_env_arr, minbound=padded_minbound, unit=envelope_lattice.unit)
print(envelope_lattice.shape)

cube_lattice = envelope_lattice.boolean_marching_cubes()

In [ ]:
facade_tiles_path1 = os.path.relpath('../data/bmc/BMC_DUMMY_256')
bmc_facademesh = bmc.marching_cube_mesh(cube_lattice, facade_tiles_path1)

In [ ]:

# initiating the plotter
p = pv.Plotter()

# # adding the meshes
p.add_mesh(tri_to_pv(bmc_facademesh), color='#638275', name="sphere", opacity=1)
# # fast visualization of the lattice
p = envelope_lattice.fast_vis(p)
# p.add_mesh(tri_to_pv(context_mesh), style='surface')
#p.add_mesh(tri_to_pv(bmc_facademesh), style ='surface',opacity=1)
# plotting
p.show(use_ipyvtk=True)